<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
!wget -c https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2023-12-04 13:32:59--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
!unzip -n concrete_data_week4.zip

Archive:  concrete_data_week4.zip


After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

### Import the necessary libraries

In [ ]:
import keras
import keras.applications
import keras.applications.vgg16
import keras.layers
import keras.models
import keras.preprocessing.image
import numpy

In [ ]:
data_generator = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.vgg16.preprocess_input,
)

In [ ]:
training_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size = (224, 224),
    batch_size = 100,
    class_mode = 'categorical',
)

Found 30001 images belonging to 2 classes.


In [ ]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (224, 224),
    batch_size = 100,
    class_mode = 'categorical',
    shuffle = False,
)

Found 9501 images belonging to 2 classes.


In [ ]:
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2
print(steps_per_epoch_training)

301


### Create the ResNet50 model

In [ ]:
resnet50_model = keras.models.Sequential()
resnet50_model.add(keras.applications.ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
))
resnet50_model.add(keras.layers.Dense(2, activation='softmax'))
resnet50_model.layers[0].trainable = False
resnet50_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet50_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23591810 (90.00 MB)
Trainable params: 4098 (16.01 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [ ]:
resnet50_model.fit(
    training_generator,
    steps_per_epoch = steps_per_epoch_training,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = steps_per_epoch_validation,
    verbose = 1,
)

Epoch 1/2
102/301 [=========>....................] - ETA: 1:03:26 - loss: 0.0219 - accuracy: 0.9951

KeyboardInterrupt: ignored

In [ ]:
resnet50_model.save('classifier_resnet_model.h5')

### Create the VGG16 model

In [ ]:
vgg16_model = keras.models.Sequential()
vgg16_model.add(
    keras.applications.VGG16(
        include_top=False,
        pooling='avg',
        weights='imagenet',
    )
)
vgg16_model.add(keras.layers.Dense(2, activation='softmax'))
vgg16_model.layers[0].trainable = False
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 14715714 (56.14 MB)
Trainable params: 1026 (4.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
fit_history = vgg16_model.fit_generator(
    training_generator,
    steps_per_epoch = steps_per_epoch_training,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = steps_per_epoch_validation,
    verbose = 1,
)

<ipython-input-16-1c585d734266>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_history = vgg16_model.fit_generator(


Epoch 1/2
 68/301 [=====>........................] - ETA: 3:24:36 - loss: 0.3988 - accuracy: 0.8174

In [ ]:
vgg16_model.save('classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (224, 224),
    batch_size = 100,
    class_mode = 'categorical',
    shuffle = False,
)

In [ ]:
resnet50_model.evaluate(validation_generator)

In [ ]:
vgg16_model.evaluate(validation_generator)

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [ ]:
resnet50_predictions = resnet50_model.predict(validation_generator)

# column 0: probability of no crack
# column 1: probability of crack
for prediction in resnet50_predictions[:5]:
    if numpy.argmax(prediction) == 0:
        print('Negative')
    else:
        print('Positive')


In [ ]:
vgg16_predictions = vgg16_model.predict(validation_generator)

# column 0: probability of no crack
# column 1: probability of crack
for prediction in vgg16_predictions[:5]:
    if numpy.argmax(prediction) == 0:
        print('Negative')
    else:
        print('Positive')

### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).